In [4]:
%load_ext autoreload
%autoreload 2
import pandas as pd 
import numpy as np
import os 
import glob
import ast 
from ast import literal_eval
from government_functions import * 
#import warnings
#warnings.formatwarning = lambda msg, *args, **kwargs: f'{msg}\n'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
final_df=pd.read_excel('results/complete_dataframe.xlsx',index_col=0)

for csv_file in glob.glob(os.path.join('government/','*.csv')): 
    file= os.path.basename(csv_file)
    countryname,_=os.path.splitext(file)
    #create country government dataframe
    df=process_gov_csv(csv_file)
    #grab part of final_df that is relevant for the country
    country_df=final_df[[col for col in final_df.columns if col.startswith(countryname)]] #this is case sensitive... 
    elections=country_df.columns.to_numpy() # list of elections in the country
    #get all election_df's (currently not really necessary but might be if stability or changes in government becomre relevant)
    election_period_dict=match_ministries_and_elections(countryname,elections,df)
    #subset election_df's for only the first ministers in an election period (initial government)
    government_dict=starting_gov_dict(election_period_dict,countryname)
    ##loop over all election periods: ##
    for date,dataframe in government_dict.items():
        if not dataframe.empty:
            party_str=country_df.at['parties',f'{countryname}{date}']
            parties=ast.literal_eval(party_str) # parses string to list  
            #create dicts
            ministry_dict,unweighted_dict,weighted_dict=get_ministry_dicts(dataframe,parties) 
            #translate dicts into lists / arrays corrosponding to the party list     
            ministy_list=[]
            unweighted_array=np.zeros(len(parties))  
            weighted_array=np.zeros(len(parties))  
            for i,(party,value) in enumerate(ministry_dict.items()): 
                ministy_list.append(value)
            for i,(party,value) in enumerate(unweighted_dict.items()):
                unweighted_array[i]=value
            for i,(party,value) in enumerate(weighted_dict.items()):
                weighted_array[i]=value
            #write country name: 
            if 'Country' not in final_df.index: 
                final_df.loc['Country']=[np.nan] * len(final_df.columns)  
            final_df.at['Country',f'{countryname}{date}']= countryname
            # write list/arrays to final_df:
            if 'Ministers' not in final_df.index: 
                final_df.loc['Ministers']=[np.nan] * len(final_df.columns)   
            if 'unweighted' not in final_df.index: 
                final_df.loc['unweighted']=[np.nan] * len(final_df.columns)   
            if 'weighted' not in final_df.index: 
                final_df.loc['weighted']=[np.nan] * len(final_df.columns) 
            final_df.at['Ministers',f'{countryname}{date}']=ministy_list
            final_df.at['unweighted',f'{countryname}{date}']=unweighted_array
            final_df.at['weighted',f'{countryname}{date}']=weighted_array 



Australia--Aug-10: Found late government.
Australia--Sep-13: Found late government.
Australia--Jul-16: Found late government.
Austria--Sep-19: Found late government.
Bulgaria--Oct-14: Found late government.
Croatia--Nov-07: Found late government.
Cyprus--May-01: Found late government.
Cyprus--May-06: Found late government.
Cyprus--May-11: Found late government.
Cyprus--May-16: Found late government.
Czech--Jun-02: Found late government.
Estonia--Mar-11: Found late government.
France--Jun-07: Found late government.
France--Jun-12: Found late government.
Germany--Sep-13: Only found 2 ministers.
Germany--Sep-13: Found late government.
Greece--Oct-09: Found late government.
Israel_99_modified--Apr-19: Only found 2 ministers.
Israel_99_modified--Sep-19: Only found 2 ministers.
Italy--Jun-87: Found late government.
Italy--Apr-96: Found late government.
Japan--Oct-96: Found late government.
Japan--Aug-09: Found late government.
Japan--Oct-17: Found late government.
Latvia--Oct-06: Found late 

Export to R 

In [3]:
rel_indices= ['mvws','Seats','Penrose-Banzhaf','Shapely-Shubik','unweighted','weighted','Country']
df=final_df.loc[rel_indices]
df=df.dropna(axis=1,how='any')
indice=['mvws','Seats','Penrose-Banzhaf','Shapely-Shubik']
df.loc[indice]=df.loc[indice].applymap(ast.literal_eval).applymap(np.array)
df.loc['mvws']=df.loc['mvws'].apply(lambda x:x/np.sum(x))
df.loc['Seats']=df.loc['Seats'].apply(lambda x:x/np.sum(x))
df.loc['unweighted']=df.loc['unweighted'].apply(lambda x:x/np.sum(x))
df.loc['weighted']=df.loc['weighted'].apply(lambda x:x/np.sum(x))


In [5]:
##exclude absolute majority parliaments: 
coal_parliaments = []
for col in df.columns: 
    mvws= df.loc['mvws',col]
    if not np.any(np.isclose(mvws,1)): 
        coal_parliaments.append(col)
df=df[coal_parliaments]

In [6]:
long_data= []
for col in final_df.columns: 
    try:
        country= df[col]['Country']
        for counting, (mvw,seats,pbs,sss,unweighted,weighted) in enumerate(zip(df[col]['mvws'],df[col]['Seats'],df[col]['Penrose-Banzhaf'],df[col]['Shapely-Shubik'],df[col]['unweighted'],df[col]['weighted'])): 
            long_data.append({'Parliament': col, 'Party': counting, 'mvw': mvw,'Seats':seats,'PB-I':pbs,'SS-I':sss,'unweighted_ministries':unweighted,'weighted_ministries':weighted,'Country':country})
    except KeyError: pass        
long_df=pd.DataFrame(long_data)

In [7]:
""" os.environ['R_HOME']="C:\Program Files\R\R-4.3.3"
import rpy2.robjects as robj
from rpy2.robjects import pandas2ri
pandas2ri.activate()
robj.globalenv['df_to_r']=long_df
 """
import pyarrow.feather as feather
feather.write_feather(long_df,"long_df.feather")

In [6]:
df = pd.read_feather('long_df.feather')

end of export 

In [ ]:
for name in list(globals()): 
    if name != 'final_df':
        del globals()[name]

## Scatter Plot Seat Share vs MVWs

In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd 
import numpy as np
import ast 
from ast import literal_eval
import matplotlib.pyplot as plt
import seaborn as sns
from government_functions import * 


In [ ]:

df=final_df.loc['mvws':'Seats']
df=df.applymap(ast.literal_eval)
df=df.applymap(np.array)
df.loc['mvws']=df.loc['mvws'].apply(lambda x:x/np.sum(x))
df.loc['Seats']=df.loc['Seats'].apply(lambda x:x/np.sum(x))
plot_data= []
for col in final_df.columns: 
    for counting, (mvw,seats) in enumerate(zip(df[col]['mvws'],df[col]['Seats'])): 
        plot_data.append({'Parliament': col, 'Party': counting, 'mvw': mvw,'Seats':seats})
plot_df=pd.DataFrame(plot_data)
    

In [ ]:
plt.figure(figsize=(10,6))
for parliament, zipped_data in plot_df.groupby('Parliament'): 
    plt.scatter(zipped_data['Seats'],zipped_data['mvw'])
plt.xlabel('Party Weights in Parliaments (Seats)')
plt.ylabel('Minimal Voting Weights (MVWs)')
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plot_df=plot_df[plot_df['mvw']<1]
for parliament, zipped_data in plot_df.groupby('Parliament'): 
    plt.scatter(zipped_data['Seats'],zipped_data['mvw'])
plt.xlabel('Party Weights in Parliaments (Seats)')
plt.ylabel('Minimal Voting Weights (MVWs)')
plt.show()

In [ ]:
rel_indices= ['mvws','Seats','unweighted','weighted']
df=final_df.loc[rel_indices]
df=df.dropna(axis=1,how='any')

In [ ]:
plot_data= []
indice=['mvws','Seats']
df.loc[indice]=df.loc[indice].applymap(ast.literal_eval).applymap(np.array)
df.loc['mvws']=df.loc['mvws'].apply(lambda x:x/np.sum(x))
df.loc['Seats']=df.loc['Seats'].apply(lambda x:x/np.sum(x))
df.loc['unweighted']=df.loc['unweighted'].apply(lambda x:x/np.sum(x))
df.loc['weighted']=df.loc['weighted'].apply(lambda x:x/np.sum(x))
for col in final_df.columns: 
    try:
        for counting, (mvw,seats,unweighted,weighted) in enumerate(zip(df[col]['mvws'],df[col]['Seats'],df[col]['unweighted'],df[col]['weighted'])): 
            plot_data.append({'Parliament': col, 'Party': counting, 'mvw': mvw,'Seats':seats,'unweighted Ministries':unweighted,'weighted Ministries':weighted})
    except KeyError: pass        
plot_df=pd.DataFrame(plot_data)
plt.figure(figsize=(10,6))
sns.scatterplot(data=plot_df, x='Seats', y='mvw')
plt.xlabel('Party Weights in Parliaments (Seats)')
plt.ylabel('Minimal Voting Weights (MVWs)')
plt.show()


In [ ]:

plot_data= []
share_in_gov=[]
for col in df.columns: 
    seats= df.loc['Seats',col]
    unweighted=df.loc['unweighted',col]
    in_gov_mask=unweighted>0 
    size_of_gov=np.sum(seats[in_gov_mask])
    share_in_gov_col=np.zeros_like(seats)
    share_in_gov_col[in_gov_mask]=seats[in_gov_mask]/size_of_gov
    share_in_gov.append(share_in_gov_col)
df.loc['% in gov',:]= share_in_gov
for col in final_df.columns: 
    try:
        for counting, (mvw,seats,unweighted,weighted,gov_share) in enumerate(zip(df[col]['mvws'],df[col]['Seats'],df[col]['unweighted'],df[col]['weighted'],df[col]['% in gov'])): 
            plot_data.append({'Parliament': col, 'Party': counting, 'mvw': mvw,'Seats':seats,'unweighted Ministries':unweighted,'weighted Ministries':weighted,'gov_share':gov_share})
    except KeyError: pass        
plot_df=pd.DataFrame(plot_data)
plot_df=plot_df[plot_df['gov_share']>0]
plot_df=plot_df[plot_df['gov_share']<1]
plot_df=plot_df[plot_df['mvw']<1]
plt.figure(figsize=(10,6))
sns.scatterplot(data=plot_df, x='gov_share', y='mvw')
plt.xlabel('Relative Party Weights in Government')
plt.ylabel('Minimal Voting Weights (MVWs) in Parliament')
plt.title('Replication Figure 1')
plt.show()

## Scatter Plot Seat Share ~ Ministries & MVW ~ Ministries

In [ ]:
plot_data= []
for col in final_df.columns: 
    try:
        for counting, (mvw,seats,unweighted,weighted) in enumerate(zip(df[col]['mvws'],df[col]['Seats'],df[col]['unweighted'],df[col]['weighted'])): 
            plot_data.append({'Parliament': col, 'Party': counting, 'mvw': mvw,'Seats':seats,'unweighted_ministries':unweighted,'weighted_ministries':weighted})
    except KeyError: pass        
plot_df=pd.DataFrame(plot_data)
plot_df.dropna(subset=['unweighted_ministries'],inplace=True)
plot_df=plot_df[plot_df['mvw']<1]
plot_df=plot_df[plot_df['unweighted_ministries']>0]
plot_df=plot_df[plot_df['weighted_ministries']<1]

In [ ]:
fig,axes= plt.subplots(2,2,figsize=(12,12))
###data:
sns.scatterplot(data=plot_df, x='mvw', y='unweighted_ministries', ax=axes[0, 0])
sns.scatterplot(data=plot_df, x='Seats', y='unweighted_ministries', ax=axes[0, 1])
sns.scatterplot(data=plot_df, x='mvw', y='weighted_ministries', ax=axes[1, 0])
sns.scatterplot(data=plot_df, x='Seats', y='weighted_ministries', ax=axes[1, 1])
### labels: 
axes[0, 0].set_title('Unweighted vs. MVW')
axes[0, 0].set_xlabel('MVW')
axes[0, 0].set_ylabel('Unweighted')

axes[0, 1].set_title('Unweighted vs. Seats')
axes[0, 1].set_xlabel('Seats')
axes[0, 1].set_ylabel('Unweighted')

axes[1, 0].set_title('Weighted vs. MVW')
axes[1, 0].set_xlabel('MVW')
axes[1, 0].set_ylabel('Weighted')

axes[1, 1].set_title('Weighted vs. Seats')
axes[1, 1].set_xlabel('Seats')
axes[1, 1].set_ylabel('Weighted')

### plot
plt.tight_layout()
plt.show()

# Regressions

2 approaches: OLS vs Logit

## OLS

See especially from the last 4 diagrams above that the relationship between Seats (or minmal voting weights) and the "score", meaning the number of ministries, seems to be generally positive and there is no reason to assume non-linearity. The data however looks to be heteroscedastic. 

I will thus later account for heteroscedastic errors:

In [ ]:
import statsmodels.api as sm 
import patsy
from patsy import dmatrices## allows "r-like" expressions to be used with statsmodels

In [ ]:
formula_unweighted_mvw='unweighted_ministries ~ mvw'
formula_weighted_mvw='weighted_ministries ~ mvw'
formula_unweighted_seats='unweighted_ministries ~ Seats'
formula_weighted_mvw='weighted_ministries ~ Seats'

In [ ]:
ols_unweighted_mvw=sm.OLS.from_formula(formula=formula_unweighted_mvw,data=plot_df)

In [ ]:
ols_unweighted_mvw_results=ols_unweighted_mvw.fit()
print(ols_unweighted_mvw_results.summary())

In [ ]:
## printing errors: 
y_matrix,x_matrix=dmatrices(formula_unweighted_mvw,plot_df,return_type='dataframe') # get design matrices
y_hat=ols_unweighted_mvw_results.predict(x_matrix) # y_hat as predicted from design matrix
#plot
plt.title('heteroscedasticity for unweighted~mvw')
plt.xlabel('y_hat')
plt.ylabel('residuals')
plt.scatter(y_hat,ols_unweighted_mvw_results.resid)
plt.show()

heteroscedasticity seems to be limited

In [ ]:
ols_unweighted_seats=sm.OLS.from_formula(formula=formula_unweighted_seats,data=plot_df)
ols_unweighted_seats_results=ols_unweighted_seats.fit()
print(ols_unweighted_seats_results.summary())

In [ ]:
## printing errors: 
y_matrix_2,x_matrix_2=dmatrices(formula_unweighted_seats,plot_df,return_type='dataframe') # get design matrices
y_hat_2=ols_unweighted_seats_results.predict(x_matrix_2) # y_hat as predicted from design matrix
#plot
plt.title('heteroscedasticity for unweighted~seats')
plt.xlabel('y_hat_2')
plt.ylabel('residuals_2')
plt.scatter(y_hat_2,ols_unweighted_seats_results.resid)
plt.show()

there seems to be more heteroscedasticity when using seat share rather than voting weights 

the following regressions correct for possibly heteroscdastic standard errors:

In [ ]:
## standard het. consistent errors: 
ols_unweighted_seats_hc0_results = ols_unweighted_seats.fit(cov_type='HC0')
print(ols_unweighted_seats_hc0_results.summary())
## het. consisitend std.errors for small samples:  
ols_unweighted_seats_hc3_results = ols_unweighted_seats.fit(cov_type='HC3')
print(ols_unweighted_seats_hc3_results.summary())

In [ ]:
ols_unweighted_mvw__hc0_results=ols_unweighted_mvw.fit(cov_type='HC0')
print(ols_unweighted_mvw__hc0_results.summary())

generally all ols specifications do not allow for any preference of voting power of seat share even for the restricted set of observations used here.